In [1]:
import numpy as np
import numpy.linalg as npla
import matplotlib.pyplot as plt


from pylgmath import Transformation, se3op
from pysteam.problem import OptimizationProblem, StaticNoiseModel, L2LossFunc, WeightedLeastSquareCostTerm
from pysteam.solver import GaussNewtonSolver
from pysteam.evaluable import se3 as se3ev
from pysteam.evaluable import Evaluable, Node, Jacobians
from pysteam.evaluable.se3 import SE3StateVar, compose_velocity, compose_rinv
from pysteam.evaluable.vspace import VSpaceStateVar

%matplotlib inline
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Times"],
    'font.size': 14,
})

np.set_printoptions(6, suppress=True)

In [2]:
class KinematicErrorEvaluator(Evaluable):
  """Evaluates radial velocity error."""

  def __init__(
      self,
      w_iv_inv: Evaluable,
  ) -> None:
    """
    Args:
      w_iv_inv: body-velocity of the moving frame (vehicle or sensor frame)
      pv: cartesian coordinate of the point in the moving frame
      r: measured radial velocity
    """
    super().__init__()

    self._w_iv_inv: Evaluable = w_iv_inv

    self._D = np.array([
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
    ])

  @property
  def active(self) -> bool:
    return self._w_iv_inv.active

  def forward(self) -> Node:
    child = self._w_iv_inv.forward()
    value = self._D @ child.value
    return Node(value, child)

  def backward(self, lhs: np.ndarray, node: Node, jacs: Jacobians) -> None:
    if self._w_iv_inv.active:
      self._w_iv_inv.backward(lhs @ self._D, node.children[0], jacs)

In [5]:
w_ms_in_s = np.loadtxt('/home/yuchen/ASRL/temp/lidar/aeva/boreas-2022-04-05-23-35/boreas-2022-04-05-23-35.kinematics_adjusted/w_mr_in_r.txt')
w_ms_in_s.shape
print(np.mean(np.abs(w_ms_in_s), axis=0))
w_ms_in_s = np.loadtxt('/home/yuchen/ASRL/temp/lidar/aeva/boreas-2022-04-05-23-35/boreas-2022-04-05-23-35.kinematics/w_mr_in_r.txt')
w_ms_in_s.shape
print(np.mean(np.abs(w_ms_in_s), axis=0))

[5.661306 0.051206 0.140473 0.035137 0.032454 0.151391]
[5.673892 0.145748 0.32053  0.040992 0.032275 0.134034]


In [ ]:
## setup trajectory
# state variables
T_vs_var = SE3StateVar(Transformation(T_ba=np.eye(4)))

## radial velocity measurement cost terms
noise_model = StaticNoiseModel(np.eye(4))
loss_func = L2LossFunc()
meas_cost_terms = []
for i in range(w_ms_in_s.shape[0]):
  w_ms_in_s_var = VSpaceStateVar(w_ms_in_s[i].reshape(6, 1), locked=True)
  w_iv_inv_eval = compose_velocity(T_vs_var, w_ms_in_s_var)
  error_func = KinematicErrorEvaluator(w_iv_inv_eval)
  meas_cost_terms.append(WeightedLeastSquareCostTerm(error_func, noise_model, loss_func))

# the problem is not well-constrained, we need to add a prior. This prior below encourages the velocity to be zero. It
#   1. penalizes translational velocity along z-axis and rotational velocity along x-axis and y-axis - since the problem is 2D
#   2. penalizes translational velocity along y-axis more than x-axis - so that we converge to the correct solution
prior_T_vs_var = SE3StateVar(Transformation(T_ba=np.eye(4)), locked=True)
noise_model = StaticNoiseModel(np.diag([1e4, 1e4, 1e4, 1e4, 1e4, 1e4]))
loss_func = L2LossFunc()
error_func = se3ev.tran2vec(compose_rinv(T_vs_var, prior_T_vs_var))
prior_cost_term = WeightedLeastSquareCostTerm(error_func, noise_model, loss_func)

opt_prob = OptimizationProblem()
opt_prob.add_state_var(T_vs_var)
opt_prob.add_cost_term(prior_cost_term, *meas_cost_terms)

gauss_newton = GaussNewtonSolver(opt_prob, verbose=True, max_iterations=100)
gauss_newton.optimize()

print(T_vs_var.value)
